In [14]:
import re

In [66]:
with open('structure_tree.txt', 'r') as f:
    lines = f.readlines()
    lines = [l.rstrip() for l in lines if len(l) > 0]
    
    current = {}
    structure_list = {}
    
    for l in lines:
        if l.startswith('          - '):
            level = 3
            text = l.strip()[2:]
        elif l.startswith('     - '):
            level = 2
            text = l.strip()[2:]
        elif l.startswith('     '):
            level = 1
            text = l.strip()
        else:
            level = 0
            text = l.strip()
            
        if len(text) == 0:
            continue
        
        try:
            res = re.findall('^(.+?)(:\s?(.*))?$', text)[0]
        except:
            print 'text ERROR:', text
            raise
            
        name, _, desc = res
        
        current[level] = name
        
        try:
            fullname, abbreviation = re.findall('^(.*),\s?(.*?)$', name)[0]
        except:
            fullname = name
            abbreviation = ''
        
        parent = current[level-1] if level > 0 else None
        
        structure_list[name] = {'level': level, 
               'fullname': fullname, 
               'abbr': abbreviation, 
               'description': desc,
              'parent': parent,
                'children': []}
        
        if parent is not None:
            structure_list[parent]['children'].append(name)
        

In [71]:
for name, s in structure_list.iteritems():
#     if s['level'] == 2:
#         print name
    print name

raphe magnus, RMg
longitudinal pontine fibers
mesencephalic tract of the trigeminal nerve, me5
medial inferior olive nucleus, IOM
dorsal lateral lemniscus, DLL
fasciculus retroflexus, fr
intermediate gray, InG
nucleus of Roller/sublingual nucleus, Ro
ventrolateral
dorsal tegmental nucleus, DTg
ambiguus nucleus, Amb
Solitary nucleus, Sol
Medulla
Pontine Reticular Formation
spinothalamic tract
Spinal trigeminal nucleus, oral part, Sp5O
central canal, CC
midbrain tegmentum
trigeminal tract
Spinal trigeminal nucleus, caudal part, Sp5C
paramedian reticular nucleus, PMn
pedunculopontine nucleus, PTg
diffuse reticular nucleus/nucleus of Kolliker-Fuse, KF
Gracile nucleus, Gr
locus coeruleus, LC
ventral tegmental nucleus, VTg
tectum
caudal pontine reticular nucleus, PnC
dorsal cochlear nucleus, DC
nucleus of the trapezoid body, Tz
oculomotor nuclear complex, 3N
Pontine Nuclei, Pn
principle sensory trigeminal nucleus, Pr5
peripeduncular nucleus, PP
supraoculomotor cap, Su3C
lateral, LVe
Central 